In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

In [19]:
from scipy import signal
def smooth(Pixels):
    win_gauss = signal.gaussian(10,3)
    smooth_Pixels = signal.convolve(Pixels, win_gauss, mode='same', method = 'direct') / win_gauss.sum()
    return(smooth_Pixels)

In [20]:
def FlowerCenteredTracks(track):
    
    strt = track.first_valid_index()
    stop = track.last_valid_index()

    x = track.loc[strt:stop, 'x'].values
    y = track.loc[strt:stop, 'y'].values

    name = [n for n in circ_parameters.name if n == mothID][0]

    circ_x = circ_parameters.loc[circ_parameters.name == name, 'circ_x'].values
    circ_y = circ_parameters.loc[circ_parameters.name == name, 'circ_y'].values
    circ_radii = circ_parameters.loc[circ_parameters.name == name, 'circ_radii'].values

    filtered_x = x 
    filtered_y = y 
    filtered_r = np.linalg.norm([filtered_x-circ_x, filtered_y-circ_y], axis = 0)
    filtered_r = filtered_r/circ_radii
    trajectory = pd.DataFrame([filtered_x, filtered_y, filtered_r]).T
    trajectory.columns = ['x', 'y', 'r']
    return(trajectory)

In [13]:
# use the list of final proboscis tracks as starting point to get the corresponding head and thorax data

visitnum = 0
direc = r"./dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/FirstVisit/"
prob_track_list = glob.glob(direc + '*.csv')

In [56]:
# get all the relevant paths and list of files

path_for_DLCAnnotation = r"G:\My Drive\Tom-Tanvi\Shared With Bing,Tom and Tanvi\Video Analysis\DeepLabCut-ImageAnalysis\take5\outputFromDLC\VideoResults\EntireDataSet/"
dlc_files = glob.glob(path_for_DLCAnnotation +  '*DeepCut_' +  '*.h5')

path_for_visit_frames = r"../MothLearning/dataFolders/Output/Step5_FilesWith_TrueTrialAnd_ProboscisDetect_v2/"

# Get moth tracking data
mothTrackPath = r"../MothLearning/dataFolders/Output/Step1_MotionDetection/"
mothTracks = glob.glob(mothTrackPath + "*.csv")

# get all the circle parameters
circ_parameters_path = glob.glob('./dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

In [67]:
len('RawDataForExplorationTime.csv')

29

In [85]:
output_fig_path = r"./dataFolders/PaperPipelineOutput/Figures/v3/OtherBodyPartTracks/FirstVisit/"
output_data = r'./dataFolders/PaperPipelineOutput/v3/OtherBodyParts/FirstVisit/' 

In [86]:
for ff in prob_track_list:
    
    a, b, _ = os.path.basename(ff).split('_')
    mothID = a + '_' + b

    # read in head data
    prob = pd.read_csv(ff)
    prob_track = prob.loc[:,'x':'y'].copy()

    # read the head data
    if not [f for f in dlc_files if mothID + 'DeepCut_' in f]:
        mothID = mothID + '_cropped'
        
    DLC_track = [f for f in dlc_files if mothID + 'DeepCut_' in f][0]
    temp = pd.read_hdf(DLC_track)

    head = temp.loc[slice(None),(slice(None),'head')]

    path_frame_Reference = glob.glob(path_for_visit_frames + mothID + '_RawDataForExplorationTime.csv')
    Visit_info_f = pd.read_csv(path_frame_Reference[0])
    Visit_info = Visit_info_f[['MothIN', 'MothOut','ProboscisDetect']]

    fin = Visit_info.iloc[visitnum, 0]
    fout = Visit_info.iloc[visitnum, 2]
    if np.isnan(fout):
        fout = Visit_info.iloc[visitnum, 1]

    fin = int(fin)
    fout = int(fout)
#     print(fin, fout)

    head_track = head[fin:fout].copy()
    head_track = head_track.droplevel(level = [0,1], axis = 1)


    # get tracks for moth for that mothID

    specificMothTracks = [f for f in mothTracks if mothID + '.csv' in f][0]

    mothTrack = pd.read_csv(specificMothTracks)
    moth_x = mothTrack.Centroid_hull_x.values[fin:fout]
    moth_y = mothTrack.Centroid_hull_y.values[fin:fout]

    filt_moth_x = smooth(moth_x)
    filt_moth_y = smooth(moth_y)
    moth_center = pd.DataFrame({'x': filt_moth_x, 'y' : filt_moth_y})

    # convert all the data to flower centric data
    proboscis = FlowerCenteredTracks(prob_track)
    headJoint = FlowerCenteredTracks(head_track)
    mothCenter = FlowerCenteredTracks(moth_center)
    
    
    f = plt.figure(figsize = (15,6))
    ax0 = f.add_subplot(131)
    ax1 = f.add_subplot(132)
    ax2 = f.add_subplot(133)
    
    ax0.plot(proboscis.r, 'm')
    ax0.plot(headJoint.r, 'b')
    ax0.plot(mothCenter.r, 'k')
    ax0.set_title('Relative Radial Position')
    ax0.set_ylabel('Relative Radial Position')

    ax1.plot(proboscis.x, 'm')
    ax1.plot(headJoint.x, 'b')
    ax1.plot(mothCenter.x, 'k')
    ax1.set_title('x position')
    ax1.set_ylabel('pixels')

    ax2.plot(proboscis.y, 'm', label = 'proboscisTip')
    ax2.plot(headJoint.y, 'b', label = 'Head-Proboscis Joint')
    ax2.plot(mothCenter.y, 'k', label = 'Moth Center of mass')
    ax2.set_title('y position')
    ax2.set_ylabel('pixels')
    ax2.legend()

    ax1.text(0.5, - 0.15, 'Frame Number (@100fps)', horizontalalignment='center',
             verticalalignment='bottom', transform=ax1.transAxes)


    plt.savefig(output_fig_path + mothID + '_FirstVisit.pdf')
    plt.close('all')
    
    proboscis.to_csv(output_data + mothID + '_proboscis.csv')
    headJoint.to_csv(output_data + mothID + '_HeadprobJoint.csv')
    mothCenter.to_csv(output_data + mothID + '_MothCOM.csv')

In [ ]:
# prob = temp.loc[slice(None),(slice(None),'proboscisTip')]

# prob_track = prob[fin:fout].copy()
# prob_track = prob_track.droplevel(level = [0,1], axis = 1)